# Ejercicio 2.4 (Tarea 2)

Un teatro utiliza un empleado para vender tiquetes y responder consultas desde las 9 a. m hasta las 5 p. m. Los puestos se adjudican únicamente si el cliente llega al teatro y paga por los tiquetes. Consultas provienen de clientes en persona o de llamadas al teatro y el empleado da prioridad a los clientes en persona. Sin embargo, gracias a un sistema complejo telefónico, las llamadas pueden esperar para ser atendidos según la política FIFO (primero en llegar primero en salir) y no renuncian hasta obtener una respuesta. Los clientes en persona llegan según una distribución exponencial con media de 12 minutos y su tiempo de servicio se distribuye exponencial con media de 6 minutos. Las llamadas ocurren según una distribución exponencial con media de 10 minutos y su tiempo de servicio sigue una distribución exponencial con media de 5 minutos. La primera persona llega a los 2 minutos y la primera llamada a los 3 minutos. Simular este sistema para un día de 8 horas y obtenga el tiempo de espera promedio de cada tipo de cliente.

## Simulación con SIMLIB/C

### Implementación SIMLIB

In [ ]:
%%writefile simlib.h
// Definiciones de tipos y constantes
#define MAX_LIST_SIZE 25 // Tamaño máximo de las listas y del arreglo transfer
#define EVENT_LIST 0     // Índice de la lista de eventos futura

// Opciones para list_file y list_remove
#define FIRST 0
#define LAST  1

// Variables globales de SIMLIB (declaradas como extern)
extern double current_time;       /* Hora actual de la simulación */
extern int next_event_type;       /* Tipo del próximo evento */
extern double transfer[MAX_LIST_SIZE + 1]; /* Arreglo para atributos de entidad */
extern int list_size[MAX_LIST_SIZE + 1]; /* Tamaño actual de cada lista */

// Prototipos de funciones de SIMLIB
void init_simlib(void);
void schedule(double time, int type);
void timing(void);
int  list_file(int option, int list);
int  list_remove(int option, int list);
double expon(double mean);
double uniform(double a, double b);
double normal(double mean, double stdev);

// Funciones para estadísticas
void init_sampst(void);
void sampst(double value, int index);
void out_sampst(FILE *f, int index);
void init_timest(void);
void timest(double value, int index);
void out_timest(FILE *f, int index);
void pr_report(FILE *f); // Para un reporte general (puede que no esté en todas las versiones)

// Funciones de utilidad (pueden variar entre implementaciones)
double ranf(void); // Generador de números aleatorios básico
void   set_random_seed(long seed);
void   init_random_number_generators(void);

Writing simlib.h


In [ ]:
%%writefile simlib.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "simlib.h"

// Definiciones de variables globales
double current_time;
int next_event_type;
double transfer[MAX_LIST_SIZE + 1];
int list_size[MAX_LIST_SIZE + 1];

// Estructura de un nodo de la lista
struct list_node {
    double attributes[MAX_LIST_SIZE + 1];
    struct list_node *next;
};

// Arreglo de punteros a la cabeza de cada lista
static struct list_node *list_heads[MAX_LIST_SIZE + 1];

// Variables para el generador de números aleatorios (RANF)
static long seed = 44444;
static double a = 16807.0;
static double m = 2147483647.0;
static double q = 127773.0;
static double r = 2836.0;

// Variables para estadísticas de sampst (muestras)
static double sampst_sum[MAX_LIST_SIZE + 1];
static double sampst_sq_sum[MAX_LIST_SIZE + 1];
static long sampst_num_observations[MAX_LIST_SIZE + 1];

// Variables para estadísticas de timest (tiempo ponderado)
static double timest_last_update_time[MAX_LIST_SIZE + 1];
static double timest_sum_area[MAX_LIST_SIZE + 1];
static double timest_value[MAX_LIST_SIZE + 1];
static long timest_num_updates[MAX_LIST_SIZE + 1];


/* init_simlib: Inicializa SIMLIB */
void init_simlib(void) {
    current_time = 0.0;
    next_event_type = 0;
    for (int i = 0; i <= MAX_LIST_SIZE; ++i) {
        list_heads[i] = NULL;
        list_size[i] = 0;
        sampst_sum[i] = 0.0;
        sampst_sq_sum[i] = 0.0;
        sampst_num_observations[i] = 0;
        timest_last_update_time[i] = 0.0;
        timest_sum_area[i] = 0.0;
        timest_value[i] = 0.0;
        timest_num_updates[i] = 0;
    }
    init_random_number_generators(); // Inicializa el generador de números aleatorios
}

/* schedule: Programa un evento */
void schedule(double time, int type) {
    struct list_node *ptr, *prev_ptr;
    struct list_node *new_node = (struct list_node *) malloc(sizeof(struct list_node));
    if (new_node == NULL) {
        fprintf(stderr, "Error: No se pudo asignar memoria para el nuevo evento.\n");
        exit(1);
    }

    new_node->attributes[1] = time; // El tiempo del evento siempre es el primer atributo
    new_node->attributes[2] = type; // El tipo de evento siempre es el segundo atributo

    // Copia los atributos adicionales de 'transfer' al nuevo nodo
    for (int i = 3; i <= MAX_LIST_SIZE; ++i) {
        new_node->attributes[i] = transfer[i];
    }

    // Inserta el evento en la lista de eventos ordenada por tiempo
    if (list_heads[EVENT_LIST] == NULL || time < list_heads[EVENT_LIST]->attributes[1]) {
        new_node->next = list_heads[EVENT_LIST];
        list_heads[EVENT_LIST] = new_node;
    } else {
        prev_ptr = list_heads[EVENT_LIST];
        while (prev_ptr->next != NULL && time >= prev_ptr->next->attributes[1]) {
            prev_ptr = prev_ptr->next;
        }
        new_node->next = prev_ptr->next;
        prev_ptr->next = new_node;
    }
    list_size[EVENT_LIST]++;
}

/* timing: Determina el próximo evento */
void timing(void) {
    struct list_node *event_node;

    if (list_heads[EVENT_LIST] == NULL) {
        fprintf(stderr, "Error: La lista de eventos está vacía. Fin de la simulación inesperado.\n");
        exit(1);
    }

    event_node = list_heads[EVENT_LIST];
    current_time = event_node->attributes[1];
    next_event_type = (int)event_node->attributes[2];

    // Copia los atributos del evento a 'transfer'
    for (int i = 1; i <= MAX_LIST_SIZE; ++i) {
        transfer[i] = event_node->attributes[i];
    }

    list_heads[EVENT_LIST] = event_node->next;
    free(event_node);
    list_size[EVENT_LIST]--;
}

/* list_file: Añade una entidad a una lista */
int list_file(int option, int list) {
    struct list_node *ptr, *prev_ptr;
    struct list_node *new_node = (struct list_node *) malloc(sizeof(struct list_node));
    if (new_node == NULL) {
        fprintf(stderr, "Error: No se pudo asignar memoria para el nuevo nodo de lista.\n");
        exit(1);
    }

    // Copia los atributos de 'transfer' al nuevo nodo
    for (int i = 1; i <= MAX_LIST_SIZE; ++i) {
        new_node->attributes[i] = transfer[i];
    }
    new_node->next = NULL;

    if (option == FIRST) { // Insertar al principio
        new_node->next = list_heads[list];
        list_heads[list] = new_node;
    } else if (option == LAST) { // Insertar al final
        if (list_heads[list] == NULL) {
            list_heads[list] = new_node;
        } else {
            ptr = list_heads[list];
            while (ptr->next != NULL) {
                ptr = ptr->next;
            }
            ptr->next = new_node;
        }
    } else {
        fprintf(stderr, "Error: Opcion invalida para list_file.\n");
        return 0; // Fallo
    }
    list_size[list]++;
    return 1; // Éxito
}

/* list_remove: Quita una entidad de una lista */
int list_remove(int option, int list) {
    struct list_node *node_to_remove;

    if (list_heads[list] == NULL) {
        // La lista está vacía, no hay nada que remover
        return 0; // Fallo
    }

    if (option == FIRST) {
        node_to_remove = list_heads[list];
        list_heads[list] = node_to_remove->next;
    } else {
        // En esta implementación simple, solo FIRST es compatible
        fprintf(stderr, "Error: Opcion invalida para list_remove. Solo FIRST es soportado en esta version.\n");
        return 0; // Fallo
    }

    // Copia los atributos del nodo removido a 'transfer'
    for (int i = 1; i <= MAX_LIST_SIZE; ++i) {
        transfer[i] = node_to_remove->attributes[i];
    }

    free(node_to_remove);
    list_size[list]--;
    return 1; // Éxito
}

/* ranf: Generador de números aleatorios de 0 a 1 */
double ranf(void) {
    long hi = seed / q;
    long lo = seed - q * hi;
    seed = a * lo - r * hi;
    if (seed < 0) {
        seed = seed + m;
    }
    return (double) seed / m;
}

/* set_random_seed: Establece la semilla del generador */
void set_random_seed(long s) {
    seed = s;
}

/* init_random_number_generators: Inicializa el generador, opcionalmente con una semilla */
void init_random_number_generators(void) {
    // Puedes llamar a set_random_seed aquí si quieres una semilla fija
    // set_random_seed(12345);
    // O puedes leer una semilla de la entrada, etc.
}

/* expon: Genera un número de una distribución exponencial */
double expon(double mean) {
    return -mean * log(ranf());
}

/* uniform: Genera un número de una distribución uniforme */
double uniform(double a_val, double b_val) {
    return a_val + (b_val - a_val) * ranf();
}

/* normal: Genera un número de una distribución normal (Box-Muller) */
double normal(double mean, double stdev) {
    static int have_spare = 0;
    static double spare;
    double u1, u2, s, val;

    if (have_spare) {
        have_spare = 0;
        return mean + stdev * spare;
    }

    do {
        u1 = ranf();
        u2 = ranf();
        s = u1 * u1 + u2 * u2;
    } while (s >= 1.0 || s == 0.0);

    val = sqrt(-2.0 * log(s) / s);
    spare = u2 * val;
    have_spare = 1;
    return mean + stdev * u1 * val;
}


/* init_sampst: Inicializa un registro de estadísticas de muestra */
void init_sampst(void) {
    // Ya inicializado en init_simlib
}

/* sampst: Registra una observación en una serie de estadísticas de muestra */
void sampst(double value, int index) {
    if (index < 1 || index > MAX_LIST_SIZE) {
        fprintf(stderr, "Error: Indice de sampst fuera de rango.\n");
        return;
    }
    sampst_sum[index] += value;
    sampst_sq_sum[index] += value * value;
    sampst_num_observations[index]++;
}

/* out_sampst: Imprime estadísticas de muestra */
void out_sampst(FILE *f, int index) {
    if (index < 1 || index > MAX_LIST_SIZE) {
        fprintf(stderr, "Error: Indice de sampst fuera de rango.\n");
        return;
    }
    fprintf(f, "Estadisticas de la muestra %d:\n", index);
    if (sampst_num_observations[index] > 0) {
        double avg = sampst_sum[index] / sampst_num_observations[index];
        double var = (sampst_sq_sum[index] - sampst_sum[index] * avg) / (sampst_num_observations[index] > 1 ? (sampst_num_observations[index] - 1) : 1);
        double std_dev = sqrt(var > 0 ? var : 0); // Evitar sqrt de negativo
        fprintf(f, "  Observaciones: %ld\n", sampst_num_observations[index]);
        fprintf(f, "  Promedio:      %.4f\n", avg);
        fprintf(f, "  Desv. Est.:    %.4f\n", std_dev);
    } else {
        fprintf(f, "  No hay observaciones.\n");
    }
}

/* init_timest: Inicializa un registro de estadísticas de tiempo ponderado */
void init_timest(void) {
    // Ya inicializado en init_simlib
}

/* timest: Actualiza una variable de estado para estadísticas de tiempo ponderado */
void timest(double value, int index) {
    if (index < 1 || index > MAX_LIST_SIZE) {
        fprintf(stderr, "Error: Indice de timest fuera de rango.\n");
        return;
    }
    // Suma el área del rectángulo anterior
    timest_sum_area[index] += timest_value[index] * (current_time - timest_last_update_time[index]);
    timest_value[index] = value; // Nuevo valor
    timest_last_update_time[index] = current_time; // Actualiza el tiempo de la última actualización
    timest_num_updates[index]++;
}

/* out_timest: Imprime estadísticas de tiempo ponderado */
void out_timest(FILE *f, int index) {
    if (index < 1 || index > MAX_LIST_SIZE) {
        fprintf(stderr, "Error: Indice de timest fuera de rango.\n");
        return;
    }
    fprintf(f, "Estadisticas de tiempo ponderado %d:\n", index);
    // Asegurarse de agregar el área del último período
    double final_sum_area = timest_sum_area[index] + timest_value[index] * (current_time - timest_last_update_time[index]);

    if (current_time > 0) {
        fprintf(f, "  Promedio: %.4f\n", final_sum_area / current_time);
    } else {
        fprintf(f, "  No hay tiempo registrado.\n");
    }
}

Writing simlib.c


### Implementación Simulación

In [1]:
%%writefile ejercicio-2-4.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "simlib.h"

// -------------------------------------------------------------------------
// Parámetros de entrada (Variables globales)
// -------------------------------------------------------------------------
#define TIEMPO_SIMULACION_HORAS 8.0
#define TIEMPO_SIMULACION_MINUTOS (TIEMPO_SIMULACION_HORAS * 60.0)

// Medias de las distribuciones exponenciales
double MEDIA_LLEGADA_CLIENTES = 12.0;
double MEDIA_SERVICIO_CLIENTES = 6.0;
double MEDIA_LLEGADA_LLAMADAS = 10.0;
double MEDIA_SERVICIO_LLAMADAS = 5.0;

// -------------------------------------------------------------------------
// Variables globales
// -------------------------------------------------------------------------
int empleado_ocupado; // 0 = libre, 1 = ocupado

// -------------------------------------------------------------------------
// Descripción del evento y tipo de evento
// -------------------------------------------------------------------------
// Eventos (tipos de eventos para la lista de eventos futura)
#define EV_LLEGADA_CLIENTE 1      // Llegada de un cliente en persona
#define EV_LLEGADA_LLAMADA 2      // Llegada de una llamada telefónica
#define EV_FIN_SERVICIO 3         // Un evento genérico para fin de servicio
#define EV_FIN_SIMULACION 4       // Fin de la simulación

// Atributos usados en el arreglo global 'transfer'
#define ATRIB_TIEMPO_LLEGADA 1  // Tiempo en que la entidad llegó a la fila (o al sistema)
#define ATRIB_TIPO_ENTIDAD 2    // Para entidades en cola: 1 = Cliente, 2 = Llamada
#define ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO 3 // Atributo para EV_FIN_SERVICIO: 1 = Cliente, 2 = Llamada

// -------------------------------------------------------------------------
// Listas y sus atributos
// -------------------------------------------------------------------------
// Listas de espera para clientes y llamadas
#define LISTA_CLIENTES 1 // Identificador para la fila de clientes en persona
#define LISTA_LLAMADAS 2 // Identificador para la fila de llamadas telefónicas

// -------------------------------------------------------------------------
// Contadores y/o acumuladores
// -------------------------------------------------------------------------
// Para estadísticas de tiempo de espera
double total_tiempo_espera_clientes = 0.0;
long num_clientes_atendidos = 0;

double total_tiempo_espera_llamadas = 0.0;
long num_llamadas_atendidas = 0;

// Índices para sampst (estadísticas de muestra)
#define INDICE_SAMPST_TIEMPO_ESPERA_CLIENTES 1
#define INDICE_SAMPST_TIEMPO_ESPERA_LLAMADAS 2

// -------------------------------------------------------------------------
// Subprogramas y propósito (Funciones para manejar los eventos)
// -------------------------------------------------------------------------

// Prototipos de funciones para el manejo de eventos
void llegada_cliente(void);
void llegada_llamada(void);
void fin_servicio(void);
void fin_simulacion(void);
void iniciar_proximo_servicio(void);

// Función principal del modelo
void model(void) {
    // Inicializar SIMLIB
    init_simlib();

    empleado_ocupado = 0; // El empleado comienza libre al inicio de la simulación

    // Programar la primera llegada de clientes y llamadas
    schedule(current_time + 2.0, EV_LLEGADA_CLIENTE); // Primera persona llega a los 2 minutos
    schedule(current_time + 3.0, EV_LLEGADA_LLAMADA);     // Primera llamada a los 3 minutos

    // Programar el evento de fin de simulación
    schedule(TIEMPO_SIMULACION_MINUTOS, EV_FIN_SIMULACION);

    // Bucle principal de la simulación
    do {
        // Obtenemos el próximo evento de la lista de eventos futura (simlib.c)
        // La información del evento se carga en el arreglo global 'transfer'.
        timing(); // Esta función actualiza 'current_time' y carga 'transfer'

        // Despachar el evento actual
        switch (next_event_type) {
            case EV_LLEGADA_CLIENTE:
                llegada_cliente();
                break;
            case EV_LLEGADA_LLAMADA:
                llegada_llamada();
                break;
            case EV_FIN_SERVICIO:
                fin_servicio();
                break;
            case EV_FIN_SIMULACION:
                break;
        }
    } while (next_event_type != EV_FIN_SIMULACION);
}

// Función para manejar la llegada de un cliente en persona
void llegada_cliente(void) {
    // Programar la próxima llegada de un cliente
    schedule(current_time + expon(MEDIA_LLEGADA_CLIENTES), EV_LLEGADA_CLIENTE);

    // Si el empleado está libre y no hay llamadas esperando (prioridad clientes)
    if (empleado_ocupado == 0 && list_size[LISTA_LLAMADAS] == 0) {
        empleado_ocupado = 1; // Ocupar empleado
        // Programar fin de servicio para este cliente
        // ¡Importante!: Cargar el atributo para el evento FIN_SERVICIO en transfer[3] (o mayor)
        transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 1; // 1 = Cliente
        schedule(current_time + expon(MEDIA_SERVICIO_CLIENTES), EV_FIN_SERVICIO);
    } else {
        // El empleado está ocupado o hay llamadas esperando, el cliente entra en la fila
        transfer[ATRIB_TIEMPO_LLEGADA] = current_time; // Registrar tiempo de llegada
        transfer[ATRIB_TIPO_ENTIDAD] = 1;             // Tipo: Cliente (para la entidad en cola)
        list_file(LAST, LISTA_CLIENTES);              // Añadir a la fila de clientes (FIFO)
    }
}

// Función para manejar la llegada de una llamada telefónica
void llegada_llamada(void) {
    // Programar la próxima llegada de una llamada
    schedule(current_time + expon(MEDIA_LLEGADA_LLAMADAS), EV_LLEGADA_LLAMADA);

    // Si el empleado está libre Y no hay clientes en la fila (prioridad clientes)
    if (empleado_ocupado == 0 && list_size[LISTA_CLIENTES] == 0) {
        empleado_ocupado = 1; // Ocupar empleado
        // Programar fin de servicio para esta llamada
        // ¡Importante!: Cargar el atributo para el evento FIN_SERVICIO en transfer[3] (o mayor)
        transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 2; // 2 = Llamada
        schedule(current_time + expon(MEDIA_SERVICIO_LLAMADAS), EV_FIN_SERVICIO);
    } else {
        // El empleado está ocupado o hay clientes esperando, la llamada entra en la fila
        transfer[ATRIB_TIEMPO_LLEGADA] = current_time; // Registrar tiempo de llegada
        transfer[ATRIB_TIPO_ENTIDAD] = 2;             // Tipo: Llamada (para la entidad en cola)
        list_file(LAST, LISTA_LLAMADAS);              // Añadir a la fila de llamadas (FIFO)
    }
}

// Función para manejar el fin de servicio para cualquier tipo de entidad
void fin_servicio(void) {
    // Recuperar el tipo de entidad que terminó el servicio
    // ¡Importante!: Ahora se lee desde el slot correcto (transfer[3]) que se guardó con el evento
    int tipo_entidad_terminado = (int)transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO];

    // Registrar que el servicio para la entidad actual ha terminado
    if (tipo_entidad_terminado == 1) { // Cliente
        num_clientes_atendidos++;
    } else if (tipo_entidad_terminado == 2) { // Llamada
        num_llamadas_atendidas++;
    } else {
        // Esto no debería pasar si la lógica es correcta
        fprintf(stderr, "Error: Tipo de entidad desconocido en fin_servicio.\n");
    }

    iniciar_proximo_servicio(); // Intentar iniciar el próximo servicio
}

// Función auxiliar para decidir qué atender a continuación
void iniciar_proximo_servicio(void) {
    // 1. Dar prioridad a los clientes en la fila
    if (list_size[LISTA_CLIENTES] > 0) {
        list_remove(FIRST, LISTA_CLIENTES); // Sacar al primer cliente de la fila
        // Calcular y registrar tiempo de espera del cliente
        total_tiempo_espera_clientes += (current_time - transfer[ATRIB_TIEMPO_LLEGADA]);
        sampst(current_time - transfer[ATRIB_TIEMPO_LLEGADA], INDICE_SAMPST_TIEMPO_ESPERA_CLIENTES);

        empleado_ocupado = 1; // Ocupar empleado
        // Programar el fin de servicio para este cliente
        // ¡Importante!: Cargar el atributo para el evento FIN_SERVICIO en transfer[3]
        transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 1; // Siguiente es un Cliente
        schedule(current_time + expon(MEDIA_SERVICIO_CLIENTES), EV_FIN_SERVICIO);
    }
    // 2. Si no hay clientes, atender llamadas
    else if (list_size[LISTA_LLAMADAS] > 0) {
        list_remove(FIRST, LISTA_LLAMADAS); // Sacar la primera llamada de la fila
        // Calcular y registrar tiempo de espera de la llamada
        total_tiempo_espera_llamadas += (current_time - transfer[ATRIB_TIEMPO_LLEGADA]);
        sampst(current_time - transfer[ATRIB_TIEMPO_LLEGADA], INDICE_SAMPST_TIEMPO_ESPERA_LLAMADAS);

        empleado_ocupado = 1; // Ocupar empleado
        // Programar el fin de servicio para esta llamada
        // ¡Importante!: Cargar el atributo para el evento FIN_SERVICIO en transfer[3]
        transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 2; // Siguiente es una Llamada
        schedule(current_time + expon(MEDIA_SERVICIO_LLAMADAS), EV_FIN_SERVICIO);
    } else {
        // Si no hay nadie en ninguna fila, el empleado queda libre
        empleado_ocupado = 0;
    }
}


int main() {
    model();

    printf("RESULTADOS SIMULACION TEATRO:\n");
    printf("\nTiempo de simulacion: %.2f horas (%.2f minutos)\n\n", TIEMPO_SIMULACION_HORAS, TIEMPO_SIMULACION_MINUTOS);
    printf("Media de tiempo entre llegadas de clientes (min): %.2f\n", MEDIA_LLEGADA_CLIENTES);
    printf("Media de tiempo de servicio de clientes (min):   %.2f\n", MEDIA_SERVICIO_CLIENTES);
    printf("Media de tiempo entre llegadas de llamadas (min): %.2f\n", MEDIA_LLEGADA_LLAMADAS);
    printf("Media de tiempo de servicio de llamadas (min):   %.2f\n", MEDIA_SERVICIO_LLAMADAS);

    printf("\nMedidas de Desempeno\n");
    // Calcular y mostrar tiempos de espera promedio
    printf("Tiempo de espera promedio de clientes en persona (min): %.2f\n", total_tiempo_espera_clientes / num_clientes_atendidos);
    printf("Tiempo de espera promedio de llamadas (min):           %.2f\n", total_tiempo_espera_llamadas / num_llamadas_atendidas);

    printf("\nNumero total de clientes atendidos: %ld\n", num_clientes_atendidos);
    printf("Numero total de llamadas atendidas: %ld\n", num_llamadas_atendidas);
    printf("Total de servicios completados: %ld\n", num_clientes_atendidos+num_llamadas_atendidas);

    printf("\nEstadisticas de Tiempos de Espera (sampst):\n");
    printf("Clientes en persona:\n");
    out_sampst(stdout, INDICE_SAMPST_TIEMPO_ESPERA_CLIENTES);
    printf("\nLlamadas telefónicas:\n");
    out_sampst(stdout, INDICE_SAMPST_TIEMPO_ESPERA_LLAMADAS);

    return EXIT_SUCCESS;
}

Writing ejercicio-2-4.c


In [ ]:
!gcc ejercicio-2-4.c simlib.c -o ejercicio-2-4 -lm
!./ejercicio-2-4

RESULTADOS SIMULACION TEATRO:

Tiempo de simulacion: 8.00 horas (480.00 minutos)

Media de tiempo entre llegadas de clientes (min): 12.00
Media de tiempo de servicio de clientes (min):   6.00
Media de tiempo entre llegadas de llamadas (min): 10.00
Media de tiempo de servicio de llamadas (min):   5.00

Medidas de Desempeno
Tiempo de espera promedio de clientes en persona (min): 7.80
Tiempo de espera promedio de llamadas (min):           18.05

Numero total de clientes atendidos: 40
Numero total de llamadas atendidas: 34
Total de servicios completados: 74

Estadisticas de Tiempos de Espera (sampst):
Clientes en persona:
Estadisticas de la muestra 1:
  Observaciones: 27
  Promedio:      11.5482
  Desv. Est.:    12.4180

Llamadas telefónicas:
Estadisticas de la muestra 2:
  Observaciones: 22
  Promedio:      27.8945
  Desv. Est.:    19.6384


#### Implementación Cambios

In [ ]:
%%writefile cambios-2-4.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "simlib.h"

// --- Parámetros de la Simulación ---
#define TIEMPO_SIMULACION_HORAS 8.0
#define TIEMPO_SIMULACION_MINUTOS (TIEMPO_SIMULACION_HORAS * 60.0)

double MEDIA_LLEGADA_CLIENTES = 12.0;
double MEDIA_SERVICIO_CLIENTES = 6.0;
double MEDIA_LLEGADA_LLAMADAS = 10.0;
double MEDIA_SERVICIO_LLAMADAS = 5.0;

// Umbral para la prioridad dinámica de llamadas
// Aunque la lógica será de "prioridad de llamadas si hay disponibles",
// mantengo el #define si quieres experimentar con umbrales más tarde.
#define UMBRAL_COLA_LLAMADAS 0 // Un umbral de 0 o 1 significa "siempre que haya una llamada"

// --- Variables de Estado del Modelo ---
int num_empleados_ocupados;
#define MAX_EMPLEADOS 2

// --- Tipos de Eventos ---
#define EV_LLEGADA_CLIENTE 1
#define EV_LLEGADA_LLAMADA 2
#define EV_FIN_SERVICIO 3
#define EV_FIN_SIMULACION 4

// --- Atributos para 'transfer' ---
#define ATRIB_TIEMPO_LLEGADA 1
#define ATRIB_TIPO_ENTIDAD 2
#define ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO 3 // Para EV_FIN_SERVICIO: 1 = Cliente, 2 = Llamada
#define ATRIB_ID_EMPLEADO_SERVICIO 4             // Nuevo: Identifica qué empleado termina el servicio (1 o 2)

// --- Identificadores de Listas ---
#define LISTA_CLIENTES 1
#define LISTA_LLAMADAS 2

// --- Contadores y Acumuladores para Estadísticas ---
double total_tiempo_espera_clientes = 0.0;
long num_clientes_atendidos = 0;

double total_tiempo_espera_llamadas = 0.0;
long num_llamadas_atendidas = 0;

// Índices para las estadísticas de muestra (sampst)
#define INDICE_SAMPST_TIEMPO_ESPERA_CLIENTES 1
#define INDICE_SAMPST_TIEMPO_ESPERA_LLAMADAS 2

// --- Prototipos de Funciones del Modelo ---
void llegada_cliente(void);
void llegada_llamada(void);
void fin_servicio(void);
void iniciar_proximo_servicio(void);
void model(void);

void model(void) {
    init_simlib();

    num_empleados_ocupados = 0;

    schedule(current_time + 2.0, EV_LLEGADA_CLIENTE);
    schedule(current_time + 3.0, EV_LLEGADA_LLAMADA);
    schedule(TIEMPO_SIMULACION_MINUTOS, EV_FIN_SIMULACION);

    do {
        timing();

        switch (next_event_type) {
            case EV_LLEGADA_CLIENTE:
                llegada_cliente();
                break;
            case EV_LLEGADA_LLAMADA:
                llegada_llamada();
                break;
            case EV_FIN_SERVICIO:
                fin_servicio();
                break;
            case EV_FIN_SIMULACION:
                break;
        }
    } while (next_event_type != EV_FIN_SIMULACION);
}

void llegada_cliente(void) {
    schedule(current_time + expon(MEDIA_LLEGADA_CLIENTES), EV_LLEGADA_CLIENTE);

    // Si hay algún empleado libre
    if (num_empleados_ocupados < MAX_EMPLEADOS) {
        num_empleados_ocupados++; // Ocupa a un empleado
        transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 1; // Servicio para un Cliente
        // No necesitamos ATRIB_ID_EMPLEADO_SERVICIO aquí, solo al finalizar.
        schedule(current_time + expon(MEDIA_SERVICIO_CLIENTES), EV_FIN_SERVICIO);
    } else {
        // Todos los empleados están ocupados, cliente va a la cola
        transfer[ATRIB_TIEMPO_LLEGADA] = current_time;
        transfer[ATRIB_TIPO_ENTIDAD] = 1; // Tipo Cliente
        list_file(LAST, LISTA_CLIENTES);
    }
}

void llegada_llamada(void) {
    schedule(current_time + expon(MEDIA_LLEGADA_LLAMADAS), EV_LLEGADA_LLAMADA);

    // Si hay algún empleado libre
    if (num_empleados_ocupados < MAX_EMPLEADOS) {
        num_empleados_ocupados++; // Ocupa a un empleado
        transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 2; // Servicio para una Llamada
        schedule(current_time + expon(MEDIA_SERVICIO_LLAMADAS), EV_FIN_SERVICIO);
    } else {
        // Todos los empleados están ocupados, llamada va a la cola
        transfer[ATRIB_TIEMPO_LLEGADA] = current_time;
        transfer[ATRIB_TIPO_ENTIDAD] = 2; // Tipo Llamada
        list_file(LAST, LISTA_LLAMADAS);
    }
}

void fin_servicio(void) {
    int tipo_entidad_terminado = (int)transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO];

    if (tipo_entidad_terminado == 1) {
        num_clientes_atendidos++;
    } else if (tipo_entidad_terminado == 2) {
        num_llamadas_atendidas++;
    } else {
        fprintf(stderr, "Error: Tipo de entidad desconocido en fin_servicio.\n");
    }

    num_empleados_ocupados--; // Un empleado se libera

    // Un empleado terminó un servicio, intenta asignar la próxima tarea
    iniciar_proximo_servicio();
    // Es importante llamar iniciar_proximo_servicio() cada vez que un empleado termina.
    // Si hay más de un empleado libre (raro, pero posible si el sistema se vacía rápido),
    // la próxima llamada a timing() y fin_servicio() liberará otro empleado y volverá a llamar.
    // Sin embargo, para asegurarnos de que se reasigne trabajo si hay múltiples empleados libres
    // y colas llenas, lo mejor es un bucle aquí.
    while (num_empleados_ocupados < MAX_EMPLEADOS && (list_size[LISTA_LLAMADAS] > 0 || list_size[LISTA_CLIENTES] > 0)) {
        iniciar_proximo_servicio();
    }
}

void iniciar_proximo_servicio(void) {
    // Solo intentar asignar si hay un empleado libre
    if (num_empleados_ocupados < MAX_EMPLEADOS) {
        // Prioridad: Llamadas primero, si hay
        if (list_size[LISTA_LLAMADAS] > 0) {
            list_remove(FIRST, LISTA_LLAMADAS);
            total_tiempo_espera_llamadas += (current_time - transfer[ATRIB_TIEMPO_LLEGADA]);
            sampst(current_time - transfer[ATRIB_TIEMPO_LLEGADA], INDICE_SAMPST_TIEMPO_ESPERA_LLAMADAS);

            num_empleados_ocupados++;
            transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 2; // Es una Llamada
            schedule(current_time + expon(MEDIA_SERVICIO_LLAMADAS), EV_FIN_SERVICIO);
        }
        // Si no hay llamadas, atender clientes si los hay
        else if (list_size[LISTA_CLIENTES] > 0) {
            list_remove(FIRST, LISTA_CLIENTES);
            total_tiempo_espera_clientes += (current_time - transfer[ATRIB_TIEMPO_LLEGADA]);
            sampst(current_time - transfer[ATRIB_TIEMPO_LLEGADA], INDICE_SAMPST_TIEMPO_ESPERA_CLIENTES);

            num_empleados_ocupados++;
            transfer[ATRIB_TIPO_ENTIDAD_EVENTO_FIN_SERVICIO] = 1; // Es un Cliente
            schedule(current_time + expon(MEDIA_SERVICIO_CLIENTES), EV_FIN_SERVICIO);
        }

    }
}

int main() {
    model();

    printf("REPORTE DE SIMULACION DE TEATRO (MEJORA: 2 EMPLEADOS)\n");
    printf("\nParametros de Entrada:\n");
    printf("  Tiempo de Simulacion: %.2f horas (%.2f minutos)\n", TIEMPO_SIMULACION_HORAS, TIEMPO_SIMULACION_MINUTOS);
    printf("  Numero de Empleados: %d\n", MAX_EMPLEADOS);
    printf("  Media Llegada Clientes: %.2f min\n", MEDIA_LLEGADA_CLIENTES);
    printf("  Media Servicio Clientes: %.2f min\n", MEDIA_SERVICIO_CLIENTES);
    printf("  Media Llegada Llamadas: %.2f min\n", MEDIA_LLEGADA_LLAMADAS);
    printf("  Media Servicio Llamadas: %.2f min\n", MEDIA_SERVICIO_LLAMADAS);
    printf("  Politica de Prioridad: Llamadas con prioridad alta (siempre que haya una en cola).\n");


    printf("\nMedidas de Desempeño:\n");
    if (num_clientes_atendidos > 0) {
        printf("  Tiempo Promedio Espera Clientes: %.2f min\n", total_tiempo_espera_clientes / num_clientes_atendidos);
    } else {
        printf("  Tiempo Promedio Espera Clientes: N/A (No clientes atendidos)\n");
    }

    if (num_llamadas_atendidas > 0) {
        printf("  Tiempo Promedio Espera Llamadas: %.2f min\n", total_tiempo_espera_llamadas / num_llamadas_atendidas);
    } else {
        printf("  Tiempo Promedio Espera Llamadas: N/A (No llamadas atendidas)\n");
    }

    printf("  Total Clientes Atendidos: %ld\n", num_clientes_atendidos);
    printf("  Total Llamadas Atendidas: %ld\n", num_llamadas_atendidas);
    printf("  Total Servicios Completados: %ld\n", num_clientes_atendidos + num_llamadas_atendidas);

    printf("\nEstadisticas Detalladas (sampst):\n");
    printf("  Tiempos de Espera Clientes:\n");
    out_sampst(stdout, INDICE_SAMPST_TIEMPO_ESPERA_CLIENTES);
    printf("  Tiempos de Espera Llamadas:\n");
    out_sampst(stdout, INDICE_SAMPST_TIEMPO_ESPERA_LLAMADAS);

    return EXIT_SUCCESS;
}

Overwriting cambios-2-4.c


In [ ]:
!gcc cambios-2-4.c simlib.c -o cambios-2-4 -lm
!./cambios-2-4

REPORTE DE SIMULACION DE TEATRO (MEJORA: 2 EMPLEADOS)

Parametros de Entrada:
  Tiempo de Simulacion: 8.00 horas (480.00 minutos)
  Numero de Empleados: 2
  Media Llegada Clientes: 12.00 min
  Media Servicio Clientes: 6.00 min
  Media Llegada Llamadas: 10.00 min
  Media Servicio Llamadas: 5.00 min
  Politica de Prioridad: Llamadas con prioridad alta (siempre que haya una en cola).

Medidas de Desempeño:
  Tiempo Promedio Espera Clientes: 3.20 min
  Tiempo Promedio Espera Llamadas: 1.65 min
  Total Clientes Atendidos: 46
  Total Llamadas Atendidas: 45
  Total Servicios Completados: 91

Estadisticas Detalladas (sampst):
  Tiempos de Espera Clientes:
Estadisticas de la muestra 1:
  Observaciones: 16
  Promedio:      9.2020
  Desv. Est.:    7.8361
  Tiempos de Espera Llamadas:
Estadisticas de la muestra 2:
  Observaciones: 15
  Promedio:      4.9374
  Desv. Est.:    3.2403


## Simulación con SimPy

In [3]:
!pip install simpy

In [8]:
import simpy
import random
import numpy as np

# --- Parámetros de la Simulación ---
TIEMPO_SIMULACION_HORAS = 8.0
TIEMPO_SIMULACION_MINUTOS = TIEMPO_SIMULACION_HORAS * 60.0

MEDIA_LLEGADA_CLIENTES = 12.0 # Media de tiempo entre llegadas de clientes en persona (minutos)
MEDIA_SERVICIO_CLIENTES = 6.0  # Media de tiempo de servicio de clientes en persona (minutos)
MEDIA_LLEGADA_LLAMADAS = 10.0     # Media de tiempo entre llegadas de llamadas (minutos)
MEDIA_SERVICIO_LLAMADAS = 5.0      # Media de tiempo de servicio de llamadas (minutos)

PRIMERA_LLEGADA_CLIENTE = 2.0  # La primera persona llega a los 2 minutos
PRIMERA_LLEGADA_LLAMADA = 3.0  # La primera llamada a los 3 minutos

# --- Variables para Resultados (Estadísticas) ---
tiempos_espera_clientes = []
tiempos_espera_llamadas = []

# --- Procesos de la Simulación ---

def cliente(env, empleado):
    """Proceso para un cliente en persona."""
    tiempo_llegada = env.now

    # Solicitar el empleado. Priority=2 (más baja que las llamadas)
    with empleado.request(priority=1) as req:
        yield req # Espera hasta que el empleado esté disponible

        tiempo_inicio_servicio = env.now
        tiempo_espera = tiempo_inicio_servicio - tiempo_llegada
        tiempos_espera_clientes.append(tiempo_espera)

        # Tiempo de servicio exponencial
        tiempo_servicio = random.expovariate(1.0 / MEDIA_SERVICIO_CLIENTES)
        yield env.timeout(tiempo_servicio) # Empleado ocupado

def llamada(env, empleado):
    """Proceso para una llamada telefónica."""
    tiempo_llegada = env.now

    # Solicitar el empleado. Priority=1 (más alta que los clientes)
    with empleado.request(priority=2) as req:
        yield req # Espera hasta que el empleado esté disponible

        tiempo_inicio_servicio = env.now
        tiempo_espera = tiempo_inicio_servicio - tiempo_llegada
        tiempos_espera_llamadas.append(tiempo_espera)

        # Tiempo de servicio exponencial
        tiempo_servicio = random.expovariate(1.0 / MEDIA_SERVICIO_LLAMADAS)
        yield env.timeout(tiempo_servicio) # Empleado ocupado


# --- Generador de Llegadas ---

def generador_llegadas_clientes(env, empleado):
    """Genera llegadas de clientes al teatro."""
    # Espera hasta la primera llegada
    yield env.timeout(PRIMERA_LLEGADA_CLIENTE)
    i = 0
    while True:
        i += 1
        env.process(cliente(env, empleado))
        tiempo_entre_llegadas = random.expovariate(1.0 / MEDIA_LLEGADA_CLIENTES)
        yield env.timeout(tiempo_entre_llegadas)

def generador_llegadas_llamadas(env, empleado):
    """Genera llegadas de llamadas telefónicas."""
    # Espera hasta la primera llegada
    yield env.timeout(PRIMERA_LLEGADA_LLAMADA)
    i = 0
    while True:
        i += 1
        env.process(llamada(env, empleado))
        tiempo_entre_llegadas = random.expovariate(1.0 / MEDIA_LLEGADA_LLAMADAS)
        yield env.timeout(tiempo_entre_llegadas)


# Configurar el entorno de simulación
env = simpy.Environment()

# Crear el empleado (recurso con capacidad 1)
empleado = simpy.PriorityResource(env, capacity=1) # Usamos PriorityResource para la prioridad

# Iniciar los generadores de llegadas
env.process(generador_llegadas_clientes(env, empleado))
env.process(generador_llegadas_llamadas(env, empleado))

# Ejecutar la simulación
random.seed(80)
env.run(until=TIEMPO_SIMULACION_MINUTOS)

# --- Reporte de Resultados ---
print("RESULTADOS SIMULACION TEATRO")
print(f"\nTiempo de Simulación: {TIEMPO_SIMULACION_HORAS} horas ({TIEMPO_SIMULACION_MINUTOS} minutos)")
print(f"Llegada Clientes (Media): {MEDIA_LLEGADA_CLIENTES} min")
print(f"Servicio Clientes (Media): {MEDIA_SERVICIO_CLIENTES} min")
print(f"Llegada Llamadas (Media): {MEDIA_LLEGADA_LLAMADAS} min")
print(f"Servicio Llamadas (Media): {MEDIA_SERVICIO_LLAMADAS} min")

promedio_espera_clientes = np.mean(tiempos_espera_clientes)
promedio_espera_llamadas = np.mean(tiempos_espera_llamadas)
print(f"\nTiempo de espera promedio de Clientes: {promedio_espera_clientes:.2f} minutos")
print(f"Tiempo de espera promedio de Llamadas: {promedio_espera_llamadas:.2f} minutos")
print(f"Total Clientes atendidos (con espera): {len(tiempos_espera_clientes)}")
print(f"Total Llamadas atendidas (con espera): {len(tiempos_espera_llamadas)}")
print(f"Total servicios completados (con espera): {len(tiempos_espera_clientes)+len(tiempos_espera_llamadas)}")

RESULTADOS SIMULACION TEATRO

Tiempo de Simulación: 8.0 horas (480.0 minutos)
Llegada Clientes (Media): 12.0 min
Servicio Clientes (Media): 6.0 min
Llegada Llamadas (Media): 10.0 min
Servicio Llamadas (Media): 5.0 min

Tiempo de espera promedio de Clientes: 13.00 minutos
Tiempo de espera promedio de Llamadas: 37.99 minutos
Total Clientes atendidos (con espera): 39
Total Llamadas atendidas (con espera): 31
Total servicios completados (con espera): 70


### Implementación Cambios

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random
import numpy as np

# --- Parámetros de la Simulación ---
TIEMPO_SIMULACION_HORAS = 8.0
TIEMPO_SIMULACION_MINUTOS = TIEMPO_SIMULACION_HORAS * 60.0

MEDIA_LLEGADA_CLIENTES = 12.0 # Media de tiempo entre llegadas de clientes en persona (minutos)
MEDIA_SERVICIO_CLIENTES = 6.0  # Media de tiempo de servicio de clientes en persona (minutos)
MEDIA_LLEGADA_LLAMADAS = 10.0     # Media de tiempo entre llegadas de llamadas (minutos)
MEDIA_SERVICIO_LLAMADAS = 5.0      # Media de tiempo de servicio de llamadas (minutos)

PRIMERA_LLEGADA_CLIENTE = 2.0  # La primera persona llega a los 2 minutos
PRIMERA_LLEGADA_LLAMADA = 3.0  # La primera llamada a los 3 minutos

# --- Variables para Resultados (Estadísticas) ---
tiempos_espera_clientes = []
tiempos_espera_llamadas = []

# --- Procesos de la Simulación ---

def cliente(env, empleado_clientes):
    """Proceso para un cliente en persona."""
    tiempo_llegada = env.now

    # Solicitar el empleado de clientes
    with empleado_clientes.request() as req:
        yield req # Espera hasta que el empleado esté disponible

        tiempo_inicio_servicio = env.now
        tiempo_espera = tiempo_inicio_servicio - tiempo_llegada
        tiempos_espera_clientes.append(tiempo_espera)

        # Tiempo de servicio exponencial
        tiempo_servicio = random.expovariate(1.0 / MEDIA_SERVICIO_CLIENTES)
        yield env.timeout(tiempo_servicio) # Empleado ocupado

def llamada(env, empleado_llamadas):
    """Proceso para una llamada telefónica."""
    tiempo_llegada = env.now

    # Solicitar el empleado de llamadas
    with empleado_llamadas.request() as req:
        yield req # Espera hasta que el empleado esté disponible

        tiempo_inicio_servicio = env.now
        tiempo_espera = tiempo_inicio_servicio - tiempo_llegada
        tiempos_espera_llamadas.append(tiempo_espera)

        # Tiempo de servicio exponencial
        tiempo_servicio = random.expovariate(1.0 / MEDIA_SERVICIO_LLAMADAS)
        yield env.timeout(tiempo_servicio) # Empleado ocupado


# --- Generadores de Llegadas ---

def generador_llegadas_clientes(env, empleado_clientes):
    """Genera llegadas de clientes al teatro."""
    # Espera hasta la primera llegada
    yield env.timeout(PRIMERA_LLEGADA_CLIENTE)
    i = 0
    while True:
        i += 1
        env.process(cliente(env, empleado_clientes))
        tiempo_entre_llegadas = random.expovariate(1.0 / MEDIA_LLEGADA_CLIENTES)
        yield env.timeout(tiempo_entre_llegadas)

def generador_llegadas_llamadas(env, empleado_llamadas):
    """Genera llegadas de llamadas telefónicas."""
    # Espera hasta la primera llegada
    yield env.timeout(PRIMERA_LLEGADA_LLAMADA)
    i = 0
    while True:
        i += 1
        env.process(llamada(env, empleado_llamadas))
        tiempo_entre_llegadas = random.expovariate(1.0 / MEDIA_LLEGADA_LLAMADAS)
        yield env.timeout(tiempo_entre_llegadas)


# Configurar el entorno de simulación
env = simpy.Environment()

# Crear los dos empleados (recursos con capacidad 1 cada uno)
empleado_clientes = simpy.Resource(env, capacity=1)
empleado_llamadas = simpy.Resource(env, capacity=1)

# Iniciar los generadores de llegadas, asignando cada uno a su empleado correspondiente
env.process(generador_llegadas_clientes(env, empleado_clientes))
env.process(generador_llegadas_llamadas(env, empleado_llamadas))

# Ejecutar la simulación
random.seed(80)
env.run(until=TIEMPO_SIMULACION_MINUTOS)

# --- Reporte de Resultados ---
print("RESULTADOS SIMULACION TEATRO")
print(f"\nTiempo de Simulación: {TIEMPO_SIMULACION_HORAS} horas ({TIEMPO_SIMULACION_MINUTOS} minutos)")
print(f"Llegada Clientes (Media): {MEDIA_LLEGADA_CLIENTES} min")
print(f"Servicio Clientes (Media): {MEDIA_SERVICIO_CLIENTES} min")
print(f"Llegada Llamadas (Media): {MEDIA_LLEGADA_LLAMADAS} min")
print(f"Servicio Llamadas (Media): {MEDIA_SERVICIO_LLAMADAS} min")
print("Configuración: Dos empleados (uno para cada cola)")

promedio_espera_clientes = np.mean(tiempos_espera_clientes)
promedio_espera_llamadas = np.mean(tiempos_espera_llamadas)
print(f"\nTiempo de espera promedio de Clientes: {promedio_espera_clientes:.2f} minutos")
print(f"Tiempo de espera promedio de Llamadas: {promedio_espera_llamadas:.2f} minutos")
print(f"Total Clientes atendidos (con espera): {len(tiempos_espera_clientes)}")
print(f"Total Llamadas atendidas (con espera): {len(tiempos_espera_llamadas)}")
print(f"Total servicios completados (con espera): {len(tiempos_espera_clientes)+len(tiempos_espera_llamadas)}")

RESULTADOS SIMULACION TEATRO

Tiempo de Simulación: 8.0 horas (480.0 minutos)
Llegada Clientes (Media): 12.0 min
Servicio Clientes (Media): 6.0 min
Llegada Llamadas (Media): 10.0 min
Servicio Llamadas (Media): 5.0 min
Configuración: Dos empleados (uno para cada cola)

Tiempo de espera promedio de Clientes: 2.50 minutos
Tiempo de espera promedio de Llamadas: 14.09 minutos
Total Clientes atendidos (con espera): 35
Total Llamadas atendidas (con espera): 52
Total servicios completados (con espera): 87
